# Lecture 9 – Exploratory Data Analysis (EDA)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.offline as py
import plotly.express as px

import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf

cf.set_config_file(offline=True, world_readable=False)

In [ ]:
# Wonhee's special methods to display large DataFrames
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
def df_interact(df, nrows=7, ncols=7):
    '''
    Outputs sliders that show rows and columns of df
    '''
    def peek(row=0, col=0):
        return df.iloc[row:row + nrows, col:col + ncols]

    row_arg = (0, len(df), nrows) if len(df) > nrows else fixed(0)
    col_arg = ((0, len(df.columns), ncols)
               if len(df.columns) > ncols else fixed(0))
    
    interact(peek, row=row_arg, col=col_arg)
    print('({} rows, {} columns) total'.format(df.shape[0], df.shape[1]))

def display_df(df, rows=pd.options.display.max_rows,
               cols=pd.options.display.max_columns):
    with pd.option_context('display.max_rows', rows,
                           'display.max_columns', cols):
        display(df)

# Introduction

In this lecture we examine the process of Exploratory Data Analysis (EDA).  Often you will acquire or even be given a collection of data in order to conduct some analysis or answer some questions. The first step in using that data is to ensure that it is in the correct form (cleaned) and that you understand its properties and limitations (EDA).  Often as you explore data through EDA you will identify additional transformations that may be required before the data is ready for analysis.

In this notebook we obtain crime data from the city of Berkeley's public records.  Ultimately, our goal might be to understand policing patterns but before we get there we must first clean and understand the data. 

# Getting the Data

To begin this analysis we want to get data about crimes in Berkeley.  Remarkably, the city of Berkeley maintains an [Open Data Portal](https://data.cityofberkeley.info/) for citizens to access data about the city.  We will be examining the:

[Call Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5)

Fortunately, this data is also relatively well document with detailed descriptions of what it contains.  Here are summaries of the fields in the data:

### Calls Data
<img src="calls_desc.png" width=800px />





## Most data has bad documentation:

Unfortunately, data is seldom well documented and when it is you may not be able to trust the documentation. It is therefore critical that when we download the data we investigate the fields and verify that it reflects the assumptions made in the documentation.



## Load the Data

In [ ]:
calls_file = "calls_for_service.csv"

# Exploring the data

Now that we have obtained the data we want to understand its:

* **Structure/Types** -- the "shape"/"type" of a data file
* **Granularity** -- how fine/coarse is each datum
* **Scope** -- how (in)complete is the data
* **Temporality** -- how is the data situated in time
* **Faithfulness** -- how well does the data capture "reality"



## Structure

Before we even begin to load the data it often helps to understand a little about the high-level structure:

1. How much data do I have?
1. How is it formatted?

### How big is the data?

I often like to start my analysis by getting a rough estimate of the size of the data.  This will help inform the tools I use and how I view the data.  If it is relatively small I might use a text editor or a spreadsheet to look at the data.  If it is larger, I might jump to more programmatic exploration or even used distributed computing tools.

However here we will use python tools to probe the file.

In [ ]:
import os
print(calls_file, "is",  os.path.getsize(calls_file) / 1e6, "MB")

All the files are relatively small and we could comfortable examine them in a text editors.  

In listing the files I noticed that the names suggest that they are all text file formats:
* **CSV**: Comma separated values is a very standard table format.

We will dive into the formats in a moment.  However because these are text data I might also want to investigate the number of lines which often correspond to records.

In [ ]:
with open(calls_file, "r") as f:
    print(calls_file, "is", sum(1 for l in f), "lines.")


### What is the file format?  (Can we trust extensions?)

We already noticed that the file end in `csv` which suggests that these is comma separated file.  However, we can't always rely on the naming as this is only a convention.  For example, here we picked the name of the file when downloading based on some hints in the URL.



**Often files will have incorrect extensions or no extension at all.**

Let's assume that these are text files (and do not contain binary encoded data) so we can print a "few lines" to get a better understanding of the file.

In [ ]:
print(calls_file, "======================")
with open(calls_file, "r") as f:
    for i in range(20):
        print(i, "\t", repr(f.readline()))

### What are some observations about `Calls` data?

1. It appears to be in comma separated value (CSV) format.
1. First line contains the column headings.
1. There are lots of **new-line** `\n` characters:
    * at the ends of lines (delimiting records?)
    * *within records* as part of addresses.
1. There are **"quoted"** strings in the `Block_Location` column:
```
"2500 LE CONTE AVE
Berkeley, CA
(37.876965, -122.260544)"
```
these are going to be difficult.  What are the implications on our earlier line count calculations?


## Loading the Data

We will now attempt to load the data into python.  We will be using the Pandas dataframe library for basic tabular data analysis.  Fortunately, the Pandas library has some relatively sophisticated functions for loading data. 


### Loading the Calls Data

Because the file appears to be a relatively well formatted CSV we will attempt to load it directly and allow the Pandas Library to deduce column headers.  (Always check that first row and column look correct after loading.)

In [ ]:
calls = pd.read_csv(calls_file, warn_bad_lines=True)
calls.head()

How many records did we get?

In [ ]:
calls.shape[0]

### Preliminary observations on the data?

1. `EVENTDT` -- Contain the incorrect time stamp
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `CVDOW` -- Appears to be some encoding of the day of the week (see data documentation).
1. `InDbDate` -- Appears to be correctly formatted and appears pretty consistent in time.
1. **`Block_Location` -- Errr, what a mess!  newline characters, and Geocoordinates all merged!!  Fortunately, this field was "quoted" otherwise we would have had trouble parsing the file. (why?)**
1. `BLKADDR` -- This appears to be the address in Block Location.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.


### Checking that the City and State fields are all Berkeley CA

We notice that there are city and state columns.  Since this is supposed to be data for the city of Berkeley these columns appear to be redundant.  Let's quickly compute the number of occurences of unique values for these two columns.

In [ ]:
calls.groupby(["City", "State"]).count()

### Decoding day of the week

According to the documentation `CVDOW=0` is Sunday, `CVDOW=1` is Monday, ...,  Therefore we can make a series to decode the day of the week for each record and join that series with the calls data.

In [ ]:
dow = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], name="Day")
dow

In [ ]:
df_dow = pd.DataFrame(dow)
# Notice that I am dropping the column if it already exists to
# make it so I can run this cell more than once
calls = pd.merge(calls.drop(columns="Day", errors="ignore"), 
         df_dow, left_on='CVDOW', right_index=True).sort_index()
calls

### Cleaning Block Location

The block location contains the latitude/longitude coordinates and I might want to use these to analyze the location of each request.  Let's try to extract the GPS coordinates using regular expressions.


In [ ]:
calls['Block_Location'].head(10)

In [ ]:
calls_lat_lon = (
    # Remove newlines
    calls['Block_Location'].str.replace("\n", "\t") 
    # Extract latitude and longitude using regular expression
    .str.extract(".*\((?P<Lat>\d*\.\d*)\, (?P<Lon>-?\d*\.\d*)\)", expand=True)
)
calls_lat_lon.head(20)

Not all the records have a lat and lon.  What fraction do have coordinates?

In [ ]:
(~calls_lat_lon.isnull()).mean()

The following block of code joins the extracted Latitude and Longitude fields with the calls data.  Notice that we actually drop these fields before joining.  This is to enable repeated invocation of this cell even after the join has been completed. 

In [ ]:
# Remove Lat and Lon if they already existed before (reproducible)
calls.drop(["Lat", "Lon"], axis=1, inplace=True, errors="ignore")
# Join in the the latitude and longitude data
calls = calls.merge(calls_lat_lon, left_index=True, right_index=True)
# calls[["Lat", "Lon"]] = calls_lat_lon
# calls.join(calls_lat_lon)
calls.head()

We can now look at a few of the records that were missing latitude and longitude entries:

In [ ]:
calls[calls['Lat'].isnull()].head(10)

Are there any patterns to the missing data?

--- 

<br/><br/><br/>

# EDA 

Now that we have loaded our various data files.  Let's try to understand a bit more about the data by examining properties of individual fields.

In [ ]:
calls.head()

---
<br/><br/><br/>

### Are Case Numbers unique?

Case numbers are probably used internally to track individual cases and my reference other data we don't have access to.  However, it is possible that multiple calls could be associated with the same case.  Let's see if the case numbers are all unique.

In [ ]:
print("There are", calls['CASENO'].unique().shape[0], "unique case numbers.")
print("There are", calls.shape[0], "calls in the table.")

Are case numbers assigned consecutively.  

In [ ]:
calls['CASENO'].sort_values().reset_index(drop=True).iplot(
    yTitle="Case Number", xTitle="Location in File")

I like to use interactive plotting tools so I can hover the mouse over the plot and read the values.  The cufflinks library adds plotly support to Pandas.  

### What might we be observing?

One possible explanation is that case numbers were assigned consecutively and then sampled uniformly at different rates for two different periods.  We will be able to understand this better by looking at the dates on the cases.

---
<br/><br/><br/>

## Examining the Date

Given the weird behavior with the case numbers let's dig into the date in which events were recorded.  Notice in this data we have several pieces of date/time information (this is not uncommon):
1. **`EVENTDT`**: This contains the date the event took place.  While it has time information the time appears to be `00:00:00`.  
1. **`EVENTTM`**: This contains the time at which the event took place.
1. **`InDbDate`**: This appears to be the date at which the data was entered in the database.  

In [ ]:
calls.head(3)

When Pandas loads more complex fields like dates it will often load them as strings:

In [ ]:
calls["EVENTDT"][0]

We will want to convert these to dates.  Pandas has a fairly sophisticated function `pd.to_datetime` which is capable of guessing reasonable conversions of dates to date objects. 

In [ ]:
dates = pd.to_datetime(calls["EVENTDT"])
dates[0]

We can verify that the translations worked by looking at a few dates:

In [ ]:
pd.DataFrame(dict(transformed=dates, original=calls["EVENTDT"])).head()

We can also extract the time field:

In [ ]:
times = pd.to_datetime(calls["EVENTTM"]).dt.time
times.head()

To combine the correct date and correct time field we use the built-in python datetime combine function.

In [ ]:
from datetime import datetime
timestamps = pd.concat([dates, times], axis=1).apply(
    lambda r: datetime.combine(r['EVENTDT'], r['EVENTTM']), axis=1)
timestamps.head()

We now updated calls to contain this additional informations:

In [ ]:
calls['timestamp'] = timestamps
calls.head()

In [ ]:
df_interact(calls)

### What time range does the data represent

In [ ]:
calls['timestamp'].min()

In [ ]:
calls['timestamp'].max()

---
<br/><br/><br/>


### Are there any other interesting temporal patterns

Do more calls occur on a particular day of the week?

In [ ]:
dow = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
calls.groupby('Day')['CASENO'].count()[dow]

In [ ]:
dow = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
calls.groupby('Day')['CASENO'].count()[dow].iplot(kind='bar', yTitle="Count")

### How about temporal patterns within a day?

In [ ]:
calls['hour_of_day'] = (
    calls['timestamp'].dt.hour * 60 + calls['timestamp'].dt.minute ) / 60.

In [ ]:
py.iplot(ff.create_distplot([calls['hour_of_day']],group_labels=["Hour"],bin_size=1, show_rug=False))

### Observations?

In the above plot we see the standard pattern of limited activity early in the morning around here 6:00AM.

---
<br/><br/><br/>

### Examining the Event

We also have data about the different kinds of crimes being reported

In [ ]:
calls.head()

### The Offense Field

The Offense field appears to contain the specific crime being reported.  As nominal data we might want to see a summary constructed by computing counts of each offense type:

In [ ]:
calls['OFFENSE'].value_counts().iplot(kind="bar")

#### Observations?

Car burglary and misdemeanor theft seem to be the most common crimes with many other types of crimes occurring rarely.

---
<br/><br/><br/>

### CVLEGEND

The CVLEGEND field provides the broad category of crime and is a good mechanism to group potentially similar crimes. 

In [ ]:
calls['CVLEGEND'].value_counts().iplot(kind="bar")

Notice that when we group by the crime time we see that **larceny** emerges as one of the top crimes.  Larceny is essentially stealing -- taking someone else stuff without force.